<h1 style="text-align:center;"><strong>Artículos sin relaciones<strong></h1>

<h4 style="text-align:justify;">Importar librerías</h4>

In [1]:
import json
import time
import pickle
import mysql.connector
import pandas as pd
import numpy as np
from ScopusModule.Client import Client
from ScopusModule.Search import Search
from ScopusModule import Utils
from sqlalchemy import create_engine
from urllib.parse import quote_plus as url_encode

In [2]:
fechaExtraccion = "13-07-2022"

<h4 style="text-align:justify;">Leer el dataframe de los artículos</h4>

In [3]:
dfArticles = pd.read_pickle('data/completo/'+fechaExtraccion+'/article.pkl')

In [4]:
dfArticles.set_index('identifier', inplace = True)

In [5]:
dfArticles

,title,publication_date,prism:doi,abstract,affiliations,authors,author_keywords,author_count,affiliation_count
identifier,,,,,,,,,
85133492759,"Metamodeling and Audio Signals Design Process,...",2023-01-01,10.1007/978-3-031-09659-4_42,The context of the following work arises from ...,"[{'afid': '60106645', 'affilname': 'Universida...","[{'authid': '57781304700', 'authname': 'Peñahe...",Audio signals design process | Experimental de...,3,2
85133293730,Facility Layout Design in Textile MSMEs. Liter...,2023-01-01,10.1007/978-3-031-09360-9_23,The capacity to respond and adapt to risks and...,"[{'afid': '60106645', 'affilname': 'Universida...","[{'authid': '57777188600', 'authname': 'Albán-...",Facility layout problem | Indicators | Resilie...,5,2
85132518705,Optimal finite sample post-selection confidenc...,2023-01-01,10.1016/j.jspi.2022.06.001,Uniformly most powerful confidence distributio...,"[{'afid': '60121225', 'affilname': 'Leuven Sta...","[{'authid': '57572669100', 'authname': 'Garcia...",Confidence distribution | Confidence interval ...,2,2
85112575431,"Corporate social responsibility, green innovat...",2022-12-19,10.1108/CR-12-2020-0160,Purpose: The purpose of this paper is to explo...,"[{'afid': '60072050', 'affilname': 'Universida...","[{'authid': '57195298226', 'authname': 'Padill...",Competitiveness | Corporate social responsibil...,2,2
85109263966,Analysing competing logics towards sustainable...,2022-12-19,10.1108/SCM-07-2020-0354,Purpose: To understand the context surrounding...,"[{'afid': '60112757', 'affilname': 'Excelia Bu...","[{'authid': '57190878471', 'authname': 'León B...",Food industry | Global supply chain | Supply c...,3,3
...,...,...,...,...,...,...,...,...,...
37049240084,Sodiro herbarium,1923-01-01,10.1126/science.58.1510.465,NaN,"[{'afid': '60032280', 'affilname': 'United Sta...","[{'authid': '23082286800', 'authname': 'Hitchc...",NaN,1,1
77957209355,The capsulín cherry: A superior form of the no...,1922-01-01,10.1093/oxfordjournals.jhered.a102156,NaN,"[{'afid': '60032280', 'affilname': 'United Sta...","[{'authid': '36511689400', 'authname': 'Popeno...",NaN,2,2
84941812731,Uncinarial nephritis,1921-01-01,10.1001/archinte.1921.00100170051003,NaN,"[{'afid': '60072042', 'affilname': 'Universida...","[{'authid': '56852172200', 'authname': 'Rojas ...",NaN,2,1


<h4 style="text-align:justify;">Obtener ids de artículos sin relaciones</h4>

In [6]:
articlesIds = dfArticles.index.tolist()

In [7]:
len(articlesIds)

40375

In [8]:
dfArticlesAffiliations = pd.read_pickle('data/completo/'+fechaExtraccion+'/article_affiliation.pkl')
dfArticlesAffiliations.head(2)

,article_id,affiliation_id
0,85133492759,60106645
1,85133293730,60106645


In [9]:
articlesIdsAffil = list(set(dfArticlesAffiliations['article_id'].tolist()))

In [10]:
len(articlesIdsAffil)

39952

In [11]:
dfArticlesAuthors = pd.read_pickle('data/completo/'+fechaExtraccion+'/article_author.pkl')
dfArticlesAuthors.head(2)

,article_id,author_id
0,85133492759,57781304700
1,85133293730,57777188600


In [12]:
articlesIdsAuth = list(set(dfArticlesAuthors['article_id'].tolist()))

In [13]:
len(articlesIdsAuth)

39952

In [14]:
set(articlesIdsAuth) == set(articlesIdsAffil) 

True

In [15]:
articlesIdsSinRelaciones = np.setdiff1d(articlesIds, articlesIdsAuth).tolist()

In [16]:
len(articlesIdsSinRelaciones)

423

In [17]:
with open('data/completo/'+fechaExtraccion+'/unrelated_articles.pkl', 'wb') as f:
    pickle.dump(articlesIdsSinRelaciones, f)

In [18]:
with open('data/completo/'+fechaExtraccion+'/unrelated_articles.pkl', 'rb') as f:
    articlesIdsSinRelaciones = pickle.load(f)

In [19]:
len(articlesIdsSinRelaciones)

423

In [20]:
for item in articlesIdsSinRelaciones:
    print('ID:', item)
    print('Titulo:', dfArticles.loc[item]['title'])
    print('Fecha de publicacion:', dfArticles.loc[item]['publication_date'])
    print('Afiliaciones:', dfArticles.loc[item]['affiliations'])
    print('Autores:', dfArticles.loc[item]['authors'], '\n')

ID: 0000187288
Titulo: The preceramic las vegas culture of coastal Ecuador
Fecha de publicacion: 1985-07-01
Afiliaciones: [{'afid': '60003212', 'affilname': 'The University of Texas at San Antonio', 'affiliation-city': 'San Antonio', 'affiliation-country': 'United States'}]
Autores: [{'authid': '6507456893', 'authname': 'Stothert K.E.', 'surname': 'Stothert', 'given-name': 'Karen E.', 'initials': 'K.E.', 'afid': ['60003212']}] 

ID: 0019107396
Titulo: Manufacture and characterization of porous polypropylene fibers
Fecha de publicacion: 1980-06-01
Afiliaciones: [{'afid': '60009165', 'affilname': 'The Dow Chemical Company', 'affiliation-city': 'Midland', 'affiliation-country': 'United States'}, {'afid': '60008279', 'affilname': 'University of Maine', 'affiliation-city': 'Orono', 'affiliation-country': 'United States'}]
Autores: [{'authid': '7103257379', 'authname': 'Andrade C.', 'surname': 'Andrade', 'given-name': 'Carlos A.', 'initials': 'C.A.', 'afid': ['60009165', '60008279']}] 

ID: 

Fecha de publicacion: 2008-01-01
Afiliaciones: [{'afid': '60095685', 'affilname': 'Albrecht-von-Haller-Institut für Pflanzenwissenschaften', 'affiliation-city': 'Gottingen', 'affiliation-country': 'Germany'}, {'afid': '60031514', 'affilname': 'Georg-August-Universität Göttingen', 'affiliation-city': 'Gottingen', 'affiliation-country': 'Germany'}]
Autores: [{'authid': '15848665600', 'authname': 'Lozada T.', 'surname': 'Lozada', 'given-name': 'Tannya', 'initials': 'T.', 'afid': ['60031514']}, {'authid': '7005136469', 'authname': 'De Koning G.H.J.', 'surname': 'De Koning', 'given-name': 'G. H.J.', 'initials': 'G.H.J.', 'afid': ['60031514']}, {'authid': '57203051832', 'authname': 'Kessler M.', 'surname': 'Kessler', 'given-name': 'Michael', 'initials': 'M.', 'afid': ['60095685']}, {'authid': '37122198500', 'authname': 'Klein A.M.', 'surname': 'Klein', 'given-name': 'Alexandra Maria', 'initials': 'A.M.', 'afid': ['60031514']}, {'authid': '7003472839', 'authname': 'Tscharntke T.', 'surname': 

Fecha de publicacion: 2012-08-01
Afiliaciones: [{'afid': '60071116', 'affilname': 'Nordic Volcanological Center', 'affiliation-city': 'Reykjavik', 'affiliation-country': 'Iceland'}, {'afid': '60032083', 'affilname': 'University at Buffalo, The State University of New York', 'affiliation-city': 'Buffalo', 'affiliation-country': 'United States'}, {'afid': '60029445', 'affilname': 'University of Minnesota Twin Cities', 'affiliation-city': 'Minneapolis', 'affiliation-country': 'United States'}, {'afid': '60018179', 'affilname': 'University of South Carolina', 'affiliation-city': 'Columbia', 'affiliation-country': 'United States'}, {'afid': '60013791', 'affilname': 'University of Hawaiʻi at Mānoa', 'affiliation-city': 'Honolulu', 'affiliation-country': 'United States'}, {'afid': '60013609', 'affilname': 'Woods Hole Oceanographic Institution', 'affiliation-city': 'Woods Hole', 'affiliation-country': 'United States'}, {'afid': '60011976', 'affilname': 'University of Alaska Fairbanks', 'affili

Autores: [{'authid': '7005718513', 'authname': 'Pacheco P.', 'surname': 'Pacheco', 'given-name': 'Pablo', 'initials': 'P.', 'afid': ['60069393']}, {'authid': '57191487294', 'authname': 'Mejía E.', 'surname': 'Mejía', 'given-name': 'Elena', 'initials': 'E.', 'afid': []}, {'authid': '56268942200', 'authname': 'Cano W.', 'surname': 'Cano', 'given-name': 'Walter', 'initials': 'W.', 'afid': ['117495636']}, {'authid': '7201947465', 'authname': 'de Jong W.', 'surname': 'de Jong', 'given-name': 'Wil', 'initials': 'W.', 'afid': ['60011001']}] 

ID: 84994614045
Titulo: Assessing the cardiology community position on transradial intervention and the use of bivalirudin in patients with acute coronary syndrome undergoing invasive management: Results of an EAPCI survey
Fecha de publicacion: 2016-10-01
Afiliaciones: [{'afid': '60170515', 'affilname': 'University Hospitals Bristol and Weston NHS Foundation Trust', 'affiliation-city': 'Bristol', 'affiliation-country': 'United Kingdom'}, {'afid': '600321

Afiliaciones: [{'afid': '60026415', 'affilname': 'Stony Brook University', 'affiliation-city': 'Stony Brook', 'affiliation-country': 'United States'}, {'afid': '60021182', 'affilname': 'Sun Yat-Sen University', 'affiliation-city': 'Guangzhou', 'affiliation-country': 'China'}, {'afid': '112614281', 'affilname': 'Guangdong Province Key Laboratory of Computational Science', 'affiliation-city': 'Guangzhou', 'affiliation-country': 'China'}]
Autores: [{'authid': '57215002551', 'authname': 'Xu Z.', 'surname': 'Xu', 'given-name': 'Zhipeng', 'initials': 'Z.', 'afid': ['60021182', '112614281', '60026415']}, {'authid': '57215006339', 'authname': 'Huang X.', 'surname': 'Huang', 'given-name': 'Xiaolong', 'initials': 'X.', 'afid': ['60026415']}, {'authid': '57214988924', 'authname': 'Jimenez F.', 'surname': 'Jimenez', 'given-name': 'Fabian', 'initials': 'F.', 'afid': []}, {'authid': '7401530888', 'authname': 'Deng Y.', 'surname': 'Deng', 'given-name': 'Yuefan', 'initials': 'Y.', 'afid': ['60026415']

<h4 style="text-align:justify;">Resultados</h4>

<table>
  <tr>
    <th>ID</th>
    <th>Razón</th>
    <th>Afiliacion</th>
  </tr>
  <tr>
    <td>0024604241</td>
    <td>Autores no tiene relación con la afiliación ecuatoriana en Scopus</td>
    <td>b Hospital General Luis Vernaza, Guayaquil, Ecuador</td>
  </tr>
  <tr>
    <td>33846834157</td>
    <td>El autor está relacionado con una afiliación ecuatoriana en Scopus pero la API lo retorna mal</td>
    <td>f IRD, CNRS, UMR 6526, Quito, Andalucia n/s, Ecuador
    </td>
  </tr>
  <tr>
    <td>85103599400</td>
    <td>Varias afiliaciones en una única afiliación y Autores no tiene relación con la afiliación ecuatoriana en Scopus</td>
    <td>Department of Mathematics and Statistics, University of Helsinki, Finland and ModeMat, Escuela Politécnica Nacional, Quito, Ecuador</td>
  </tr>
  <tr>
    <td>85105065131</td>
    <td>Varias afiliaciones en una única afiliación</td>
    <td>a Instituto de Estudios Urbanos y Territoriales, Pontificia Universidad Católica de Chile, Santiago Centro de Desarrollo Urbano Sustentable, Pontificia Universidad Católica de Chile, Santiago Facultad de Arquitectura y Urbanismo, Universidad UTE, Quito, Ecuador</td>
  </tr>
</table>
